<a href="https://colab.research.google.com/github/LGLV/APS-Failure-at-Scania-Trucks-Data-Set/blob/main/7_2_Funci%C3%B3n_de_similitud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Matriz de similitud***

---
---

Última actualización: 28-02-2025.

---
---

# ***Tabla de Contenido***
La mayoría de los puntos en este script se emplearon en el estudio de librerias quimioenfocadas.
- [0 - ***Importar librerias***](#0)
- [1 - ***Lectura de base de datos*** ](#1)
- [2 - ***Selección de función de similitud*** ](#2)
- [3 - ***Gráfico de identidad*** ](#3)
- [4 - ***Referencias*** ](#4)

<a name='0'></a>
# **Librerias: instalar e importar**

In [ ]:
from IPython.utils import io
import tqdm.notebook
import os, os.path, sys, random, subprocess
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
        pbar.update(20)
        #Librerias gráficas
        !pip install matplotlib
        import matplotlib.pyplot as plt
        import matplotlib.font_manager as font_manager
        %matplotlib inline
        import time

        !pip install seaborn
        import seaborn as sns
        pbar.update(40)
        #Librerias del sistema y herramientas primarias:
        import pandas as pd
        import warnings
        warnings.filterwarnings("ignore")
        %config Completer.use_jedi = False
        pbar.update(30)
        #Busqueda
        !pip install rdkit
        from rdkit import Chem, DataStructs
        from rdkit.DataStructs import TanimotoSimilarity, DiceSimilarity, CosineSimilarity
        from rdkit.Chem import AllChem, Draw, PandasTools, Descriptors, Descriptors3D, rdMolDescriptors, Scaffolds
        from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol

        PandasTools.RenderImagesInAllDataFrames(images=True) #Para visualizar moléculas
        from rdkit.Chem.Draw import IPythonConsole
        import math
        from math import pi
        from pathlib import Path
        from tempfile import TemporaryDirectory
        !pip install numpy
        import numpy as np
        # Mount Google Drive and upload your PyMOL license
        from google.colab import drive
        drive.mount("/content/drive")
        pbar.update(10)

<a name='1'></a>
# **1 - Lectura de base de datos**

In [ ]:
url="https://raw.githubusercontent.com/DIFACQUIM/Cursos/main/Datasets//08_Similitud_compounds_g9a.csv"
id=pd.read_csv(url,index_col=[0])
id.head()

In [ ]:
id.sort_values(by=["document_chembl_id"], ascending=False, inplace=True)
id = id.drop_duplicates(subset='molecule_chembl_id')
# Reset index
id.reset_index(drop=True, inplace=True)
print(id.shape)
id[0:3]

<a name='2'></a>
# **2 - Selección de función de similitud**

In [ ]:

def similarity_calc(smi1, smi2, radius, method='tanimoto'):
    """
    Calcula la similitud entre dos moléculas dadas sus representaciones SMILES.

    Parámetros:
    - smi1 (str): SMILES de la primera molécula.
    - smi2 (str): SMILES de la segunda molécula.
    - radius (int): Radio para la generación de ECFP.
    - method (str): Método de similitud ('tanimoto', 'dice', 'cosine', 'euclidean').

    Retorna:
    - similarity (float): Valor de similitud calculado.
    """

    # Generador de huellas ECFP
    fpgen = AllChem.GetMorganGenerator(radius=radius, fpSize=2048)

    # Convertir SMILES a moléculas
    mol1 = Chem.MolFromSmiles(smi1)
    mol2 = Chem.MolFromSmiles(smi2)

    if mol1 is None or mol2 is None:
        raise ValueError("Una de las moléculas no se pudo cargar correctamente. Verifique los SMILES.")

    # Generar las huellas moleculares
    fp1 = fpgen.GetFingerprint(mol1)
    fp2 = fpgen.GetFingerprint(mol2)

    # Calcular la similitud según el método elegido
    if method == 'tanimoto':
        similarity = round(TanimotoSimilarity(fp1, fp2), 3)

    elif method == 'dice':
        similarity = round(DiceSimilarity(fp1, fp2), 3)

    elif method == 'cosine':
        similarity = round(CosineSimilarity(fp1, fp2), 3)

    elif method == 'euclidean':
        # Convertir las huellas a arrays y calcular la distancia Euclidiana
        arr1 = np.array(fp1)
        arr2 = np.array(fp2)
        similarity = round(np.linalg.norm(arr1 - arr2), 3)

    else:
        raise ValueError("Método no reconocido. Use 'tanimoto', 'dice', 'cosine' o 'euclidean'.")

    return similarity

In [ ]:
#Tomar una muestra aleatoria
id_sample = id.sample(n=100, random_state=1)

tic =time.time()
hmap=pd.DataFrame()
for i in id_sample.index:
    for j in id_sample.index:
        a=id_sample.loc[i].at['smiles']
        b=id_sample.loc[j].at['smiles']
        alignment= similarity_calc(a,b, 3, 'tanimoto')
        hmap.loc[id_sample.loc[i,'molecule_chembl_id'],id_sample.loc[j,'molecule_chembl_id']]=alignment
toc =time.time()
print("Time: "+ str((toc-tic))+" s")
#Matriz de 414x414, con tiempo de ejecución aproximado de: 5 min

In [ ]:
#hmap.to_excel('8_compound_aligment.xlsx')
hmap

<a name='1'></a>
# **3 - Gráfico de identidad**

In [ ]:
palette= sns.blend_palette(['black', 'blue','orange'],74)
palette

In [ ]:
fig, ax = plt.subplots(figsize=(40,40))
sns.heatmap(hmap,square = True, xticklabels=hmap.columns, yticklabels=hmap.index, cmap=palette, vmin=0, vmax=1,
            cbar_kws={'shrink':0.8,
                      'orientation':'horizontal','spacing':'uniform',
                      'pad':0.02})
ax.figure.axes[1].tick_params(axis="x", labelsize=28) #Tamaño de ticks en cbar
ax.figure.axes[1].set_xlabel('Coeficiente de tamimoto', size=28, weight='bold') #Etiqueta de cbar
ax.tick_params(labelsize=28)
plt.xticks (np.arange(len(hmap.index))+0.5, hmap.index,rotation=90)
plt.yticks (np.arange(len(hmap.columns))+0.5, hmap.columns, rotation=0)
ax.xaxis.tick_top()
plt.tick_params ('both',width=1,labelsize=5)
#plt.savefig('Coeficiente_tanimoto.png',dpi=300,format='png',transparent=False)
plt.show()

<a name='4'></a>
# **4 - Referencias:**

1.   https://projects.volkamerlab.org/teachopencadd/talktorials/T001_query_chembl.html
2.   https://gist.github.com/leelasd/746f3c447805fc6ea5aad1071bd77aef

*   https://github.com/rdkit/UGM_2014/blob/master/Notebooks/Scaffold%20analysis%20of%20ChEMBL%20data%20with%20pandas%20and%20RDKit/Scaffold%20analysis%20of%20ChEMBL%20data%20with%20pandas%20and%20RDKit%20-%20RDKit%20UGM2014.ipynb
*   https://github.com/rdkit/UGM_2013/blob/master/Tutorials/pandastools/Pandas_RDKit_UGM.ipynb
*https://github.com/rdkit/UGM_2013/blob/master/Tutorials/pandastools/Pandas_RDKit_UGM.ipynb

